In [92]:
import numpy as np
import math
from scipy.optimize import fsolve

def truncate(f, n):
    return math.floor(f * 10 ** n) / 10 ** n

def cal_TDlambda(proba=0.5, r=None, v=None, _lambda=0.5):
    assert r is not None
    assert v is not None
    
    E1 = (1-_lambda)*(
        v[0] + proba*(r[0]+v[1]-v[0]) + (1-proba)*(r[1]+v[2]-v[0])
    )
    E2 = _lambda*(1-_lambda)*(
        v[0] + proba*(r[0]+r[2]+v[3]-v[0]) + (1-proba)*(r[1]+r[3]+v[3]-v[0])
    )
    E3 = _lambda**2*(1-_lambda)*(
        v[0] + proba*(r[0]+r[2]+r[4]+v[4]-v[0]) + (1-proba)*(r[1]+r[3]+r[4]+v[4]-v[0])
    )
    E4 = _lambda**3*(1-_lambda)*(
        v[0] + proba*(r[0]+r[2]+r[4]+r[5]+v[5]-v[0]) + (1-proba)*(r[1]+r[3]+r[4]+r[5]+v[5]-v[0])
    )
    E5 = _lambda**4*(1-_lambda)*(
        v[0] + proba*(r[0]+r[2]+r[4]+r[5]+r[6]+v[6]-v[0]) + (1-proba)*(r[1]+r[3]+r[4]+r[5]+r[6]+v[6]-v[0])
    )
    Einf = _lambda**5*(
        v[0] + proba*(r[0]+r[2]+r[4]+r[5]+r[6]) + (1-proba)*(r[1]+r[3]+r[4]+r[5]+r[6])
    )
    
    return E1 + E2 + E3 + E4 + E5 + Einf

v = [0.0,16.8,0,24,17.3,-4.6,6.5]
r = [2.4,-1.9,-2.9,5.9,2.2,7.4,-4.3]
proba = 0.39

val = cal_TDlambda(proba=proba, v=v, r=r, _lambda=1)
solution = fsolve(lambda x: cal_TDlambda(proba=proba, v=v, r=r, _lambda=x) - val, np.linspace(0.1,1,10))

solution = np.sort(solution)
for s in solution:
    if s>0 and s<1:
        print(truncate(s,3))
        break

0.562


In [70]:
val

23.526

In [82]:
cal_TDlambda(proba=proba, v=v, r=r, _lambda=0.4956)

5.278207853874127

In [64]:
def getTD1(probToState1, valueEstimates, rewards):
    VS6 = 0
    VS5 = VS6 + rewards[6]
    VS4 = VS5 + rewards[5]
    VS3 = VS4 + rewards[4]
    
    VS2 = VS3 + rewards[3]
    VS1 = VS3 + rewards[2]
    
    VS0 = probToState1 * (VS1 + rewards[0]) + (1 - probToState1) * (VS2 + rewards[1])
    
    return VS0

def getE1(probToState1, valueEstimates, rewards):
    VS1 = valueEstimates[1]
    VS2 = valueEstimates[2]
    
    VS0 = probToState1 * (VS1 + rewards[0]) + (1 - probToState1) * (VS2 + rewards[1])
    
    return VS0

def getE2(probToState1, valueEstimates, rewards):
    VS3 = valueEstimates[3]
    VS1 = rewards[2] + VS3
    VS2 = rewards[3] + VS3
    
    VS0 = probToState1 * (VS1 + rewards[0]) + (1 - probToState1) * (VS2 + rewards[1])
    
    return VS0

def getE3(probToState1, valueEstimates, rewards):
    VS4 = valueEstimates[4]
    VS3 = rewards[4] + VS4
    VS1 = rewards[2] + VS3
    VS2 = rewards[3] + VS3
    
    VS0 = probToState1 * (VS1 + rewards[0]) + (1 - probToState1) * (VS2 + rewards[1])
    
    return VS0
def getE4(probToState1, valueEstimates, rewards):
    VS5 = valueEstimates[5]
    VS4 = rewards[5] + VS5
    VS3 = rewards[4] + VS4
    VS1 = rewards[2] + VS3
    VS2 = rewards[3] + VS3
    
    VS0 = probToState1 * (VS1 + rewards[0]) + (1 - probToState1) * (VS2 + rewards[1])
    
    return VS0
def getE5(probToState1, valueEstimates, rewards):
    VS6 = valueEstimates[6]
    VS5 = rewards[6] + VS6
    VS4 = rewards[5] + VS5
    VS3 = rewards[4] + VS4
    VS1 = rewards[2] + VS3
    VS2 = rewards[3] + VS3
    
    VS0 = probToState1 * (VS1 + rewards[0]) + (1 - probToState1) * (VS2 + rewards[1])
    
    return VS0
def getE6(probToState1, valueEstimates, rewards):
    VS6 = 0 + 0 # no reward no value from "S7"
    VS5 = rewards[6] + VS6
    VS4 = rewards[5] + VS5
    VS3 = rewards[4] + VS4
    VS1 = rewards[2] + VS3
    VS2 = rewards[3] + VS3
    
    VS0 = probToState1 * (VS1 + rewards[0]) + (1 - probToState1) * (VS2 + rewards[1])
    
    return VS0

def getEstimators(probToState1, valueEstimates, rewards):
    E1 = getE1(probToState1, valueEstimates, rewards)
    E2 = getE2(probToState1, valueEstimates, rewards)
    E3 = getE3(probToState1, valueEstimates, rewards)
    E4 = getE4(probToState1, valueEstimates, rewards)
    E5 = getE5(probToState1, valueEstimates, rewards)
    E6 = getE6(probToState1, valueEstimates, rewards)
    
    print((E1, E2, E3, E4, E5, E6))
    return (E1, E2, E3, E4, E5, E6)
def findLambda(probToState1, valueEstimates, rewards):
    E = getEstimators(probToState1, valueEstimates, rewards)

    coeffs = [E[5] - E[4], E[4] - E[3], E[3] - E[2], E[2] - E[1], E[1] - E[0], E[0] - E[5]]

    print(np.roots(coeffs))

In [65]:
findLambda(proba, v, r)

(-1.0479999999999998, 28.326, 22.126, 14.325999999999999, 23.526, 11.225999999999999)
[-1.16015001+0.j         0.20622303+1.3010633j  0.20622303-1.3010633j
  1.        +0.j         0.49567142+0.j       ]
